# gp

> Fill in a module description here

In [ ]:
#| default_exp gp

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from sepia.SepiaModel import SepiaModel
from sepia.SepiaData import SepiaData
import pkg_resources
import os
from EarlyDarkEmu.pca import do_pca
import numpy as np
from EarlyDarkEmu.load import sepia_data_format


In [ ]:
#| export

MODEL_DIR = "model/"
MODEL_FILE = pkg_resources.resource_stream("EarlyDarkEmu", MODEL_DIR + "README.txt").name
MODEL_DIR_PATH = os.path.dirname(MODEL_FILE)

In [ ]:
#| export

def do_gp_train(sepia_model:SepiaModel=None, # Input data in SEPIA format, after PCA
                model_file:str=None, #pickle file path
               ) -> SepiaModel: # sepia.SepiaModel.SepiaModel after GP
    
    ## Also check: https://github.com/lanl/SEPIA/blob/master/examples/Synthetic_toy_examples/multivariate_example_with_prediction.ipynb
    
    sepia_model.tune_step_sizes(50, 20, update_vals=True)
    sepia_model.do_mcmc(1000)
    sepia_model.save_model_info(model_file)
    return sepia_model

In [ ]:
#| export

def gp_load(sepia_model:SepiaModel=None, # Input data in SEPIA format (Pre-PCA is fine? -- CHECK)
            model_file:str=MODEL_DIR_PATH + 'multivariate_model', #pickle file path
    ) -> SepiaModel: # sepia.SepiaModel.SepiaModel
    
    # make sure this model was instantiated with the same input data as the model corresonding to this saved model info.

    # if os.path.isfile(model_file+'.pkl'):
    try:
        sepia_model.restore_model_info(model_file)
        return sepia_model
        
    except FileNotFoundError as e:
        print(e.errno)    

In [ ]:
#| export

def gp_load_all():
    pass

In [ ]:
#| export 

def do_gp_train_multiple(model_dir:str=None, # Pickle directory path
                         p_train_all:np.array=None, # Parameter array
                         y_vals_all:np.array=None, # Target y-values array
                         y_ind_all:np.array=None, # x-values
                         z_index_range:np.array=None, # Snapshot indices for training
                   ) -> None: 
    
    for z_index in z_index_range:
     
        sepia_data = sepia_data_format(p_train_all, y_vals_all[:, z_index, :], y_ind_all)
        # print(sepia_data)
        model_filename = model_dir + 'multivariate_model_z_index' + str(z_index) 

        sepia_model = do_pca(sepia_data, exp_variance=0.95)
        sepia_model = do_gp_train(sepia_model, model_filename)
        print('Training complete for snapshot ' + str(z_index))
        print('Model saved at ' + model_dir)
        print(30*'=*')
    
    return None

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()